# 垃圾回收机制

没有垃圾回收机制会导致程序不能释放不再使用的内存(内存泄露)。

## 引用计数

In [2]:
import os
import psutil
def show_memory(hint):
    pid=os.getpid()
    p=psutil.Process(pid)
    info =p.memory_full_info()
    memeory=info.uss/1024.0/1024
    print("{} memory used:{} MB".format(hint,memeory))

def func():
    show_memory('before a created')
    a=[i for i in range(100000)]
    show_memory('after a created')

func()
show_memory('end')

AccessDenied: psutil.AccessDenied (pid=20728)

列表a创建后，内存占用变大，当`func`函数执行完毕后，内存又恢复正常水平，这是因为列表a是局部变量，函数执行完毕后，引用a被销毁，a指向的列表对象的引用计数为0，Python进行垃圾回收，回收a指向的列表对象。可以通过`sys.getrefcount()`获取引用计数。

In [8]:
import sys
dd=[]
sys.getrefcount(dd) # 一个引用来自dd，getrefcount的参数也会引用

2

In [10]:
def f(a):# 函数调用时，会产生两次引用，函数栈和函数参数
    print(sys.getrefcount(a))
f(dd)
print(sys.getrefcount(dd))

4
2


还能手动启动垃圾回收。

In [11]:
import gc
del dd# 删除引用
gc.collect()# 回收内存

23780

In [12]:
a=1123
d=a
del d
a

1123

## 循环引用

如果发生循环引用，引用计数将不起作用。

In [14]:
def func():
#     show_memory('init')
    a=[i for i in range(100000)]
    b=[i for i in range(100000)]
#     show_memory('created')
    a.append(b)
    b.append(a)

func()
# show_memory('finished')# 不会回收

In [15]:
gc.collect()

3591

Python针对循环引用，使用标记清除算法和分代收集来执行自动垃圾回收。
- 标记清除算法：通过一个节点去遍历所有的对象，并标记途径的对象，遍历结束后，没有被标记的对象，将被来及回收。每次执行遍历，性能浪费很大，Python采用双向链表维护了一个数据结构，并且只考虑容器类的对象(只有容器类对象才会产生循环引用)。
- 分代收集：分代收集是一个优化手段，所有的对象被分为3代，刚刚创建的为0代，经历一次垃圾回收，任然存在的对象将从上一代挪到下一代。每一代的自动启动垃圾回收的阈值可以设置，当到达阈值后，将自动启动这一代的垃圾回收(0代的对象更可能被垃圾回收)。

## 内存泄露调试

`objgraph`模块，可视化显示引用关系：
- show_refs()：引用关系图。
- show_backrefs():